In [1]:
import networkx as nx
import pickle
import random
import scipy
from scipy.sparse import tril, find
import django
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
import igraph
from django.core.management import call_command

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from cities.models import *


In [23]:
q = Query.objects.get(pk=7965)

pd.set_option('display.max_colwidth', -1)
dis = []
pat = "[a-z|A-Z]{2} [0-9]{5}"
for d in q.doc_set.all():
    if d.docauthinst_set.exists():
        fai = d.docauthinst_set.order_by('position').first().institution
        if fai=="" or fai==[""] or fai=="['']":
            continue
        cstring = fai.split(',')[-1]
        country = None
        cstring = cstring.replace('.','')
        if "USA" in cstring:
            country = Country.objects.get(name="United States")
        if country is None:
            if "Trinid & Tobago" in cstring:
                country = Country.objects.get(name="Trinidad and Tobago")
        if country is None:
            for ukc in ["england","scotland","wales","northern Ireland"]:
                if ukc in cstring.lower():
                    country = Country.objects.get(name="United Kingdom") 
                    
        if country is None:
            if "DEM REP CONGO" in cstring:
                country = Country.objects.get(name__icontains="Democratic Republic of the Congo")
            if "Cote Ivoire" in cstring:
                country = Country.objects.get(name__icontains="Ivory Coast")
                
        if country is None:
            if "Mongol Peo Rep".lower() in cstring.lower():
                country = Country.objects.get(name="Mongolia")

        if country is None:
            for c in Country.objects.all():
                if c.name.lower() in cstring.lower() or c.name.lower() in cstring.replace('U ','United ').lower():
                    country = c
                    break
        if country is None:
            for c in Country.objects.all():
                for alt in c.alt_names.all():
                    if alt.name.lower() in cstring.lower():
                        country = c
                        break
                        
                        
        if country is None:
            if re.match(pat, cstring):
                country = Country.objects.get(name="United States")
            if "washington" in fai.lower():
                country = Country.objects.get(name="United States")
            
        if country is None:
            print(cstring, fai)
            continue
                
        dis.append({"id":d.id, "fai": fai,"country": country.name, "continent": country.continent.name})
    else:
        print(d.title)
df = pd.DataFrame.from_dict(dis)

df.head()

Health benefits of policies to mitigate climate change
IEA - World energy outlook 2004
International greenhouse gas emissions budgets and joint implementation
 Byelarus Belarusian State Univ, Minsk 220050, Byelarus.
Land-Based Pollution in the Arctic Ocean: Canadian Actions in a Regional and Global Context
 Fr Polynesia CNRS, EPHE, UMS 2978, Ctr Rech Insulaires, Papetoai, Moorea, Fr Polynesia.
 Papua New PNG Univ Technol, Dept Agr, Lae 411, Morobe Province, Papua New.
 Byelarus IPNASB, 68 Nezavisimosti Ave, Minsk 220072, Byelarus.
Reducing global carbon dioxide emission
Antagonism, Neo-liberalism and Movements: Six Impossible Things Before Breakfast
Heatwaves blamed on global warming (vol 488, pg 143, 2012)
 * Univ Virginia, Dept Civil & Environm Engn, *, *, *.
Health, Safety and Environment as the Basis for Innovative Education on Sustainability
Water Challenges and Solutions on a Global Scale
A Case Study: Still Water Electrical Generation
New York City Panel on Climate Change 2015 R

,continent,country,fai,id
0,Europe,United Kingdom,"Liverpool John Moores Univ, Res Ctr Evolutionary Anthropol & Palaeoecol, Sch Nat Sci & Psychol, Liverpool L3 3AF, Merseyside, England.",1095
1,Europe,Austria,"Leopold Franzens Univ Innsbruck, Inst Bot, A-6020 Innsbruck, Austria.",1133
2,North America,United States,"Arizona State Univ, Sch Sustainabil, Tempe, AZ 85287 USA.",1135
3,North America,United States,"Columbia Univ, Dept Econ, 1022 Int Affairs Bldg,MC 3308,420 West 118th St, New York, NY 10027 USA.",1152
4,Europe,Poland,"Panstwowy Inst Badawczy, Inst Ochrony Srodowiska, Warsaw, Poland.",1164


In [20]:
pat = "[a-z|A-Z]{2} [0-9]{5}"
cstring = "MA 02138"
re.match(pat,cstring)

<_sre.SRE_Match object; span=(0, 8), match='MA 02138'>

In [22]:
Country.objects.get(name="Mongolia").alt_names.values('name')

<QuerySet [{'name': 'Mongolian People’s Republic'}, {'name': 'Mongol Peoples Republic'}, {'name': 'Mongolia'}]>